Human Powered Retriever
---

## 環境構築

ライブラリのインストール。

In [ ]:
%%capture
%pip install langchain openai tiktoken

環境変数 `MY_OPENAI_API_KEY` に使用する API キーを設定しておきましょう。

In [ ]:
import os
os.environ["OPENAI_API_KEY"] = os.getenv("MY_OPENAI_API_KEY")

## 関数

### get_knowledge_string

In [ ]:
def get_knowledge_string():
    """
    data/ ディレクトリ以下のファイルをすべて読み込んで, 一つのテキストにして返す
    ファイルごとに、ファイル名を文頭にわかりやすく追加する。

    Returns:
        str: テキスト
    """
    import os
    import glob

    files = glob.glob('data/**', recursive=True)
    files.sort()
    text = ''

    for file in files:
        if os.path.isdir(file):
            continue
        with open(file, 'r') as f:
            text += '---\n'
            text += f'# {file}\n\n'
            text += f.read()
            text += '\n\n'

    print(f"文字列の長さ: {len(text)}")

    import tiktoken
    encoding = tiktoken.encoding_for_model("gpt-4-1106-preview")
    tokens = len(encoding.encode(text))
    print(f"トークン数: {tokens}\n")

    if tokens >= 120000:
        raise Exception('トークン数が120000を超えています。')

    return text

### generate_prompt

In [ ]:
def generate_prompt(question: str):
    """
    LLM に入力するプロンプトを生成する

    Args:
        question (str): 質問

    Returns:
        str: プロンプト
    """
    prompt = f"""
    {get_knowledge_string()}


    上記のテキストを参考に質問に答えてください。

    Q: {question}
    """
    return prompt

## メイン

### データを削除

In [ ]:
remove_data = False

In [ ]:
if remove_data:
    import os
    import glob
    files = glob.glob('data/**', recursive=True)
    for file in files:
        if os.path.isdir(file):
            continue
        os.remove(file)

In [ ]:
remove_data = True

### Download ディレクトリからファイルを移動

In [ ]:
import_download = False

In [ ]:
if import_download:
    print('ダウンロードディレクトリの md ファイルを読み込みます')
    !mkdir -p ./data
    !mv ~/Downloads/*.md ./data/
    !ls ./data

In [ ]:
import_download = True

### 質問

In [ ]:
question_enabled = False

In [ ]:
question = f"""
GKE の料金体系について簡単に教えてください
"""

if question_enabled and question.strip() != '':
    from langchain.chat_models import ChatOpenAI
    chat = ChatOpenAI(model='gpt-4-1106-preview')
    print(chat.predict(generate_prompt(question)))

In [ ]:
question_enabled = True